In [85]:
import tensorly as tl
from tensorly.decomposition import parafac, partial_tucker
import numpy as np
import torch
import torch.nn as nn
from VBMF import VBMF

# SVD
参考：https://zhuanlan.zhihu.com/p/42777652

In [86]:
from numpy.linalg import svd
import numpy as np

# 根据我们设置的百分比计算出最少需要的奇异值的数量
def select_K(sigma, percentage):
    square = sigma**2 
    base = sum(square) 
    s = 0 
    k = 0
    for i in sigma:
        s += i**2
        k += 1
        if s >= base * percentage:
            return k

data = np.array([[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]])

u, sigma, v = svd(data)
'''
这里得到的
U和v是左右奇异向量矩阵
sigma是奇异值列表，并且从大到小排序
如果data的尺寸是m*n
那么，
U尺寸是m*m
V尺寸是n*n
'''

#找出压缩的奇异值数量
k = select_K(sigma, 0.98) # 5
sigmaK = np.mat(np.eye(k) * sigma[:k]) 

# 最后压缩之后得到的是item的矩阵，其中的每一个行向量对应一个item。
itemMat = np.matmul(np.matmul(data.T,u[:,:k]), sigmaK.I)
print("U: ", u[:,:k].shape)
print("sigmaK: ", sigmaK.shape)
print("V: ", v[:k,:].shape)
print("itemMat: ",itemMat.shape)
#==========================================================

# 重建数据
# data_compress=u[:, :k] * sigma[:k] @ v[:k, :]
data_compress = u[:, :k] @ sigmaK @ v[:k, :]
print(data_compress)

U:  (11, 5)
sigmaK:  (5, 5)
V:  (5, 11)
itemMat:  (11, 5)
[[-6.35987013e-02 -1.75000038e-01 -2.11189866e-01  6.63243807e-02
  -8.38366511e-03  3.94935046e+00 -7.54167361e-03  1.87194064e-01
   2.19645050e-01 -3.63779257e-03  5.00657873e+00]
 [ 3.69312900e-03  4.23362282e-02  5.42305142e-02  3.10727215e+00
   1.16611626e-02  3.65499402e+00 -1.49379599e-02 -4.14369054e-02
  -7.16127377e-02 -8.41207652e-03  3.27922702e+00]
 [ 1.53123341e-01  1.21783621e-01  1.57541798e-01 -4.12181011e-02
   4.19151330e+00  7.34300163e-03  3.53701064e-02  4.31482341e-01
   1.63228657e-01  3.83012245e+00  2.19472453e-02]
 [ 2.81884646e+00  2.28120460e+00  2.94905331e+00  2.61664828e-01
  -2.84987225e-02 -1.29973230e-01 -2.52257574e-02  2.87537670e+00
   2.90863719e+00 -2.92679341e-02 -4.53969065e-02]
 [ 4.87540590e+00  3.92479612e+00  5.07578890e+00  7.21898915e-02
  -2.02526648e-03 -1.86878944e-01 -1.12568960e-02  5.01377306e+00
   5.07653945e+00  2.08579249e-03  1.41553672e-01]
 [-6.86636747e-02 -6.042341

## weight,factors = tl.decomposition.parafac(randint_tensor, rank=rank)

In [87]:
import numpy as np
import tensorly as tl

# 创建一个形状为 (2, 3) 的随机整数数组，在 [low, high) 范围内
randint_tensor = np.random.randint(low=1, high=10, size=(4, 5,6,7))
# print(randint_tensor)

# 执行CP分解，假设分解的秩为2
weight,factors = tl.decomposition.parafac(randint_tensor, rank=5)

# 打印分解得到的因子
for factor in factors:
    print(factor,factor.shape)

[[69.73197575 -1.65361332 -4.96177232 -0.5023591   3.34929773]
 [75.08698228 -2.76731812 -1.09477499  3.25216814  0.55126361]
 [71.78454884  0.10877523  1.44494359  2.72581091  1.34335892]
 [72.38644068 -5.85877685 -1.63931796 -0.54552514  3.13910576]] (4, 5)
[[ 0.45476539 -0.09749693 -0.65080847  0.41220718  0.31728844]
 [ 0.4376696   0.65803641  0.24957269 -0.60565502  0.8189591 ]
 [ 0.47142717  0.88353733 -0.26320738 -0.12378585 -0.71448683]
 [ 0.44776154 -0.23253811  0.64680698  0.50702296  0.44195478]
 [ 0.41076839 -0.03321074  0.90737748  0.89789492  1.76992413]] (5, 5)
[[ 0.40994823 -0.87027073  0.19115964 -0.92631294  0.77614502]
 [ 0.42130478 -1.08900854  0.61445631 -1.06850321 -0.14368451]
 [ 0.36650137 -0.22304109  1.06141629  1.00004339  0.64244921]
 [ 0.4050568  -0.30503273  1.14770903  0.68642186 -0.11907894]
 [ 0.41537085  0.65192425  0.20687916 -0.23592464  0.61394999]
 [ 0.42873761 -1.22519892 -0.10441571  0.13504315  0.29708115]] (6, 5)
[[ 0.33901505  0.61893537 -0.10

## cp分解的api：tl.decomposition.parafac
外积重建

In [126]:
import tensorly as tl
import numpy as np

# 创建一个示例四维张量
tensor_shape = (4,5,6,7)
original_tensor = np.random.rand(*tensor_shape)

# 执行CP分解，假设分解的秩为2
rank = 4
weight,factors = tl.decomposition.parafac(original_tensor, rank=rank)
print(weight)

#=============================================================================
# 重建张量
reconstructed_tensor = tl.cp_to_tensor((weight,factors))

# 计算原始张量与重建张量的差异
difference = np.linalg.norm(original_tensor - reconstructed_tensor)

# 打印差异
print(f"原始张量与重建张量的差异：{difference}")

# 打印原始张量和重建张量的形状
print("原始张量的形状：", original_tensor.shape)
print("重建张量的形状：", reconstructed_tensor.shape)


# 手动重建张量
reconstructed_tensor = np.zeros(tensor_shape)
for i in range(rank):
    # last_two=np.outer(factors[2][:, i], factors[3][:, i])
    # last_three=np.outer(factors[1][:, i],last_two)
    # last=np.outer(factors[0][:, i],last_three).reshape(factors[0].shape[0],factors[1].shape[0],factors[2].shape[0],factors[3].shape[0])

    # 对每个模式的因子进行外积
    factor_product = np.outer(factors[0][:, i], np.outer(factors[1][:, i], np.outer(factors[2][:, i], factors[3][:, i]))).reshape(factors[0].shape[0],factors[1].shape[0],factors[2].shape[0],factors[3].shape[0])
    # 累加到重建张量
    reconstructed_tensor += factor_product

# 计算原始张量与重建张量的差异
difference = np.linalg.norm(original_tensor - reconstructed_tensor)

# 打印差异
print(f"原始张量与重建张量的差异：{difference}")

# 打印原始张量和重建张量的形状
print("原始张量的形状：", original_tensor.shape)
print("重建张量的形状：", reconstructed_tensor.shape)

[1. 1. 1. 1.]
原始张量与重建张量的差异：7.62643178862702
原始张量的形状： (4, 5, 6, 7)
重建张量的形状： (4, 5, 6, 7)
原始张量与重建张量的差异：7.62643178862702
原始张量的形状： (4, 5, 6, 7)
重建张量的形状： (4, 5, 6, 7)


# 张量的Kruskal 形式  

Matrix可以表示为两个向量的外积和，三阶张量表示为三个向量的外积和,  $N$ 阶段张量可以表示为N个向量的外积和. 和的个数表示为张量的 Kruskal 秩.  

![kruskal_tensor](../../示例图片/triplets.png)  


## 获得 Kruskal 表示: CP 分解  
使用CP对张量进行秩 $R$ 分解.  

在TensorLy库中只需要简单调用函数 **parafac**:

In [56]:
import tensorly as tl
import torch

# 分解卷积矩阵
def cp_decomposition_conv_layer(layer, rank):
    """ Gets a conv layer and a target rank, 
        returns a nn.Sequential object with the decomposition """

    # Perform CP decomposition on the layer weight tensorly. 
    # last, first, vertical, horizontal = \
    weight, (last, first, vertical, horizontal) = \
        tl.decomposition.parafac(layer.weight.data.numpy(), rank=rank, init='svd')

    # print(weight)
    
    print('conv2d layer shape: ',layer.weight.data.numpy().shape)
    print('rank: ', rank)

    print('first shape: ',first.shape)
    print('vertical shape: ',vertical.shape)
    print('horizontal shape: ',horizontal.shape)
    print('last shape: ',last.shape)

    pointwise_s_to_r_layer = torch.nn.Conv2d(
        in_channels=first.shape[0],
        out_channels=first.shape[1], 
        kernel_size=1, 
        stride=1, 
        padding=0, 
        dilation=layer.dilation, 
        bias=False
        )

    depthwise_vertical_layer = torch.nn.Conv2d(
        in_channels=vertical.shape[1], 
        out_channels=vertical.shape[1], 
        kernel_size=(vertical.shape[0], 1),
        stride=1, 
        padding=(layer.padding[0], 0), 
        dilation=layer.dilation,groups=vertical.shape[1], 
        bias=False
        )

    depthwise_horizontal_layer = torch.nn.Conv2d(
        in_channels=horizontal.shape[1], 
        out_channels=horizontal.shape[1], 
        kernel_size=(1, horizontal.shape[0]), 
        stride=layer.stride,
        padding=(0, layer.padding[0]), 
        dilation=layer.dilation, 
        groups=horizontal.shape[1], 
        bias=False
        )

    pointwise_r_to_t_layer = torch.nn.Conv2d(
        in_channels=last.shape[1], 
        out_channels=last.shape[0], 
        kernel_size=1, 
        stride=1,
        padding=0, 
        dilation=layer.dilation, 
        bias=True
        )

    print('pointwise_s_to_r_layer shape: ', pointwise_s_to_r_layer.weight.data.numpy().shape)
    print('depthwise_vertical_layer shape: ', depthwise_vertical_layer.weight.data.numpy().shape)
    print('depthwise_horizontal_layer shape: ', depthwise_horizontal_layer.weight.data.numpy().shape)
    print('pointwise_r_to_t_layer shape: ', pointwise_r_to_t_layer.weight.data.numpy().shape)
    print('################################################################')

    pointwise_r_to_t_layer.bias.data = layer.bias.data

    #这里给每个层的参数提供数据，来自cp分解的因子
    #(last, first, vertical, horizontal)对应(kernel_nums,channel_nums,higth_kernel_size,width_kernel_size)
    depthwise_horizontal_layer.weight.data = torch.transpose(torch.tensor(horizontal), 1, 0).unsqueeze(1).unsqueeze(1)

    depthwise_vertical_layer.weight.data = torch.transpose(torch.tensor(vertical), 1, 0).unsqueeze(1).unsqueeze(-1)

    pointwise_s_to_r_layer.weight.data = torch.transpose(torch.tensor(first), 1, 0).unsqueeze(-1).unsqueeze(-1)

    pointwise_r_to_t_layer.weight.data = torch.tensor(last).unsqueeze(-1).unsqueeze(-1)

    new_layers = [pointwise_s_to_r_layer, depthwise_vertical_layer, depthwise_horizontal_layer, pointwise_r_to_t_layer]
    
    return nn.Sequential(*new_layers)


In [4]:

def estimate_ranks(layer):
    """ Unfold the 2 modes of the Tensor the decomposition will 
    be performed on, and estimates the ranks of the matrices using VBMF 
    """

    weights = layer.weight.data
    unfold_0 = tl.base.unfold(weights.numpy(), 0) 
    unfold_1 = tl.base.unfold(weights.numpy(), 1)
    _, diag_0, _, _ = VBMF.EVBMF(unfold_0)
    _, diag_1, _, _ = VBMF.EVBMF(unfold_1)
    ranks = [diag_0.shape[0], diag_1.shape[1]]
    return ranks


# Tucker 分解

Tucker 分解可以看成CP分解的一般形式. 在卷积中实际只要对ic/oc进行分解即可，相当于Trucker分解有用的性质是，它不必沿着所有的轴（模）来分解。我们可以沿着输入和输出通道进行分解（模2的分解） 

![tucker_tensor](../../示例图片/tucker.png)  


In [5]:

def tucker_decomposition_conv_layer(layer):
    """ Gets a conv layer, 
        returns a nn.Sequential object with the Tucker decomposition.
        The ranks are estimated with a Python implementation of VBMF
        https://github.com/CasvandenBogaard/VBMF
    """

    print('conv2d layer shape: ',layer.weight.data.numpy().shape)
    ranks = estimate_ranks(layer)
    print(layer, "VBMF Estimated ranks", ranks)
    core, [last, first] = \
        partial_tucker(layer.weight.data.numpy(), \
            modes=[0, 1], rank=ranks, init='svd')

    print('first shape: ',first.shape)
    print('core shape: ',core.shape)
    print('last shape: ',last.shape)

    # A pointwise convolution that reduces the channels from S to R3
    first_layer = torch.nn.Conv2d(in_channels=first.shape[0], \
            out_channels=first.shape[1], kernel_size=1,
            stride=1, padding=0, dilation=layer.dilation, bias=False)

    # A regular 2D convolution layer with R3 input channels 
    # and R3 output channels
    core_layer = torch.nn.Conv2d(in_channels=core.shape[1], \
            out_channels=core.shape[0], kernel_size=layer.kernel_size,
            stride=layer.stride, padding=layer.padding, dilation=layer.dilation,
            bias=False)

    # A pointwise convolution that increases the channels from R4 to T
    last_layer = torch.nn.Conv2d(in_channels=last.shape[1], \
        out_channels=last.shape[0], kernel_size=1, stride=1,
        padding=0, dilation=layer.dilation, bias=True)

    print('first_layer shape: ', first_layer.weight.data.numpy().shape)
    print('core_layer shape: ', core_layer.weight.data.numpy().shape)
    print('last_layer shape: ', last_layer.weight.data.numpy().shape)
    print('################################################################')

    last_layer.bias.data = layer.bias.data

    first_layer.weight.data = \
        torch.transpose(torch.tensor(first.copy()), 1, 0).unsqueeze(-1).unsqueeze(-1)
    last_layer.weight.data = torch.tensor(last.copy()).unsqueeze(-1).unsqueeze(-1)
    core_layer.weight.data = torch.tensor(core.copy())

    new_layers = [first_layer, core_layer, last_layer]
    return nn.Sequential(*new_layers)


In [6]:

# VGG16 based network for classifying between dogs and cats.
# After training this will be an over parameterized network,
# with potential to shrink it.
class ModifiedVGG16Model(torch.nn.Module):
    def __init__(self, model=None):
        super(ModifiedVGG16Model, self).__init__()

        model = models.vgg16(pretrained=True)
        self.features = model.features

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(25088, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 2))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

class LeNet(torch.nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.25),
            nn.Linear(9216, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        output = F.log_softmax(x, dim=1)
        return output


In [7]:
model = torch.load("/home/huan.wang/lessons/fastai-course/chapter2/mnist_model").cuda()
print(model)
model.eval()
model.cpu()
N = len(model.features._modules.keys())
for i, key in enumerate(model.features._modules.keys()):
    if i >= N - 2: # 后续fc层
        break
    if isinstance(model.features._modules[key], torch.nn.modules.conv.Conv2d):
        conv_layer = model.features._modules[key]
        rank = max(conv_layer.weight.data.numpy().shape)//3
        decomposed = cp_decomposition_conv_layer(conv_layer, rank)
        model.features._modules[key] = decomposed

torch.save(model, 'decomposed_mnist_model_cp')


LeNet(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.25, inplace=False)
    (1): Linear(in_features=9216, out_features=128, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)
conv2d layer shape:  (32, 1, 3, 3)
rank:  10
first shape:  (1, 10)
vertical shape:  (3, 10)
horizontal shape:  (3, 10)
last shape:  (32, 10)
pointwise_s_to_r_layer shape:  (10, 1, 1, 1)
depthwise_vertical_layer shape:  (10, 1, 3, 1)
depthwise_horizontal_layer shape:  (10, 1, 1, 3)
pointwise_r_to_t_layer shape:  (32, 10, 1, 1)
################################################################
conv2d layer shape:  (64, 32, 3, 3)
rank:  21
fi

In [8]:
model = torch.load("/home/huan.wang/lessons/fastai-course/chapter2/vgg_model").cuda()
print(model)
model.eval()
model.cpu()
N = len(model.features._modules.keys())
for i, key in enumerate(model.features._modules.keys()):
    if i >= N - 2: # 后续fc层
        break
    if isinstance(model.features._modules[key], torch.nn.modules.conv.Conv2d):
        conv_layer = model.features._modules[key]
        decomposed = tucker_decomposition_conv_layer(conv_layer)
        model.features._modules[key] = decomposed

torch.save(model, 'decomposed_vgg_model_tucker')

ModifiedVGG16Model(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, paddin